# Fake News Detection

## Importing the libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 


## Importing the dataset

In [6]:
true_df=pd.read_csv("E:\Machine Learning Tutorials\Fake-News-Detection\data\True.csv")
fake_df=pd.read_csv('E:\Machine Learning Tutorials\Fake-News-Detection\data\Fake.csv')

## Modify the Dataset 

In [7]:
#Checking the columns
print(true_df.columns)
print(fake_df.columns)


Index(['title', 'text', 'subject', 'date'], dtype='object')
Index(['title', 'text', 'subject', 'date'], dtype='object')


In [8]:
#ADD True/False column to the dataframes
true_df['True/Fake']=1
fake_df['True/Fake']=0




In [9]:
#Dropping the irrelevant columns
true_df.drop(['text','subject','date'],axis=1, inplace=True)
fake_df.drop(['text','subject','date'],axis=1, inplace=True)
print(true_df.columns)
print(fake_df.columns)

Index(['title', 'True/Fake'], dtype='object')
Index(['title', 'True/Fake'], dtype='object')


## Merge and Shuffle the datasets

In [10]:
#Merge
dataset=pd.concat([true_df,fake_df],axis=0,ignore_index=True)



In [11]:
dataset.head()

,title,True/Fake
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [12]:
dataset.tail()

,title,True/Fake
44893,McPain: John McCain Furious That Iran Treated ...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
44896,How to Blow $700 Million: Al Jazeera America F...,0
44897,10 U.S. Navy Sailors Held by Iranian Military ...,0


In [13]:
#Shuffling the dataset
dataset=dataset.sample(frac=1,random_state=69).reset_index(drop=True)


In [14]:
dataset.head()

,title,True/Fake
0,Downfall of ex-Samsung strategy chief leaves '...,1
1,"Attorney General Sessions visits White House, ...",1
2,Trump Has Finally Commented On Portland Train...,0
3,Lawmaker says U.S. foreign surveillance 'unmas...,1
4,SHOCKING POLL RESULTS In Primary Victories Ton...,0


In [15]:
dataset.tail()

,title,True/Fake
44893,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,0
44894,Turkey's Erdogan defends Ottoman commander aft...,1
44895,BREAKING: Dems Celebrate Progressive Win As D...,0
44896,"Trump bashes Clinton, promises not to become b...",1
44897,Democrats want Congress' authorization for ant...,1


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      44898 non-null  object
 1   True/Fake  44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.7+ KB


## Cleaning the texts

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(0, len(dataset)):
    title = re.sub('[^a-zA-Z]', ' ', dataset['title'][i])
    title = title.lower()
    title = title.split()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')  # Keep "not" if needed
    title = [lemmatizer.lemmatize(word) for word in title if word not in set(all_stopwords)]
    title = ' '.join(title)
    corpus.append(title)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [18]:
print(corpus[0:10])

['downfall ex samsung strategy chief leaf salarymen disillusioned', 'attorney general session visit white house not trump', 'trump finally commented portland train attack sort tweet', 'lawmaker say u foreign surveillance unmasked trump associate', 'shocking poll result primary victory tonight', 'difficult chelsea clinton cause high turnover clinton foundation', 'tina fey rachel maddow go head head tonight show video', 'never guess conservative blowhard compared trump hitler video', 'obama sanction russia intervening election', 'florida mother allows baby bitten snake laugh sick viral video']


## Creating the Bag of Words model

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [20]:
len(X[0])

3000

## Splitting the dataset into the Training set and Test set

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [23]:
pred_lr=LR.predict(X_test)


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, pred_lr)
print(cm)
accuracy_score(y_test, pred_lr)

[[4422  306]
 [ 195 4057]]


0.9442093541202673

##  Naive Bayes 

In [25]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [26]:
pred_NB = classifier.predict(X_test)
print(np.concatenate((pred_NB.reshape(len(pred_NB),1), y_test.reshape(len(y_test),1)),1))

[[0 1]
 [1 1]
 [0 0]
 ...
 [1 1]
 [1 1]
 [1 1]]


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, pred_NB)
print(cm)
accuracy_score(y_test, pred_NB)

[[3856  872]
 [ 219 4033]]


0.8785077951002227

## Decision Tree

In [28]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion ='entropy' ,random_state = 0)
classifier.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [29]:
pred_NB = classifier.predict(X_test)
print(np.concatenate((pred_NB.reshape(len(pred_NB),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [0 0]
 ...
 [1 1]
 [1 1]
 [1 1]]


In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, pred_NB)
print(cm)
accuracy_score(y_test, pred_NB)

[[4353  375]
 [ 492 3760]]


0.9034521158129176